In [20]:
# my version
from stemming.porter2 import stem
import nltk
import os
import bisect # increase insert efficiency
# store the stop words as set.
with open("englishST.txt","r") as file:
    STOPWORDS=set(file.read().splitlines())

In [ ]:
def preprocessing(filename):
    file_content = open(filename).read()
    
    tokens = nltk.word_tokenize(file_content) # tokenisation
    # case folding(convert text into lower case)
    # remove stop words
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in STOPWORDS]
    tokens = [stem(token) for token in tokens] # normalisation:stem
    filename = 'output_'+filename[:-4]+'.txt'
    f = open(filename, "a")
    f.write("\n".join(tokens))
    f.close()

In [99]:
# return stemmed_words is a list of tuples(word, docID, position)
def preprocess(text, document_number):
    # tokenisation
    tokens = nltk.word_tokenize(text) 
    
    # case folding(convert text into lower case)
    # remove non-letter and non-digit
    tokens = [token.lower() for token in tokens if token.isalpha() or token.isdigit()]
    
#     # remove english stopwords
#     tokens = [token for token in tokens if token not in STOPWORDS]
    
    # # normalisation:stemming
    stemmed_words = [stem(token) for token in tokens]

    words_and_positions = list(
                    zip(stemmed_words, [document_number] * len(stemmed_words), range(1, len(stemmed_words) + 1)))

    # stemmed_words is a list of tuples(word, docID, position)
    # where elem = word, docID
    return words_and_positions

In [10]:
import xml.etree.ElementTree as ElementTree
import re
from stemming.porter2 import stem
import math
import pickle

# xml file to text version
def xmlTotext(filename):
    tree = ElementTree.parse(filename)
    root = tree.getroot()
    filename = filename[:-4] + '_output' + '.txt'
    fil = open(filename, "a")
    for childs in root:
        for i in range(len(childs)):
            if childs[i].tag == 'DOCNO':
                fil.write("ID: " + childs[i].text.strip()+'\n')
            elif childs[i].tag == 'HEADLINE':
                fil.write("HEADLINE: "+ childs[i].text.strip()+'\n')
            elif childs[i].tag == 'TEXT':
                fil.write("TEXT: "+ childs[i].text.strip()+'\n')
    fil.close()

In [102]:
postings = dict()
term_frequency = dict()


{}


In [ ]:
def add_posting(document_number, position):

    if document_number in postings.keys():
        bisect.insort(postings[document_number], position)
        self.term_frequency[document_number] += 1
    else:
        self.postings[document_number] = [position]
        self.term_frequency[document_number] = 1
        self.add_appearance()

In [ ]:
def build_inverted_index(self, input_file):
    """
    :param input_file: original XML file
    :return: dictionary of words and information about them, such as the document frequency and postings
    """

    with open(input_file, 'r') as f:  # Reading file
        xml = f.read()

        xml = '<ROOT>' + xml + '</ROOT>'  # Let's add a root tag

        root = ElementTree.fromstring(xml)
        word_list = []

        # Simple loop through each document
        for doc in root:
            for elem in doc:

                if elem.tag.lower() == 'docno':
                    document_number = int(elem.text.strip())


                elif elem.tag.lower() == 'headline':
                    headline = elem.text.strip()

                    # preprocess the headline
                    preprocessed_headline = self.preprocess(headline, document_number)
                    word_list.extend(preprocessed_headline)

                elif elem.tag.lower() == 'text':
                    text = elem.text.strip()

                    # preprocess the text
                    # this returns a list of tuples
                    # (word, document_number, position)
                    # positions counted after stopword removal
                    preprocessed_text = self.preprocess(text, document_number)
                    word_list.extend(preprocessed_text)

    # sort the word list by word
    # then by document number
    # appearances should already be ordered
    
    
    word_list = sorted(word_list)

    # merge into postings list
    for elem in word_list:
        word = elem[0]
        document_number = elem[1]
        position = elem[2]

        new_term = Term(word)

        if word in self.index.keys():
            # the index has seen this word before
            # we update the number of occurrences
            # and the postings list
            term = self.index[word]
            term.add_posting(document_number, position)

        else:
            # print("Word not in index!")
            new_term.add_posting(document_number, position)
            self.index[word] = new_term

    return self.index